In [20]:
# !pip install gdown
# !pip install emoji

In [21]:
# !pip install sentencepiece

In [22]:
import csv
from google.colab import files
from google.colab import drive
import pandas as pd
import re
import unicodedata

In [23]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
path='/content/drive/MyDrive/Unigoto/merged_data.csv'


In [25]:
merged_df = pd.read_csv(path, low_memory=False)

In [26]:
df = merged_df.copy()

In [27]:
# удаляем колонку "deactivated"
df = df.drop(columns=['deactivated'], errors='ignore')

In [28]:
filtered_data = df[df['country_title'].isin(['Россия'])]

In [29]:
# # определяем паттерн для поиска русских городов
# pattern = re.compile(r'[^a-zA-Z0-9\s]')
# filtered_data = filtered_data[filtered_data['city_title'].str.contains(pattern, na=True)]

In [30]:
# удаление строк, где пропущено больше двух значений из столбцов
columns_to_check = ['about', 'activities', 'books', 'games', 'interests']
filtered_data = filtered_data.dropna(subset=columns_to_check, thresh=len(columns_to_check)-2)
# удаление строк, где пропущена основная целевая метка
filtered_data = filtered_data.dropna(subset=['university_name'])

In [31]:
# заполнение пропущенных значений mode() или Unknown, используя группировку
filtered_data['faculty_name'] = filtered_data.groupby(['university_name', 'faculty'])['faculty_name'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'Unknown'))

In [49]:
columns_to_drop = ['graduation', 'country_id', 'country_title', 'city_id', 'university', "faculty", "city_title"]
filtered_data = filtered_data.drop(columns=columns_to_drop, errors='ignore', axis=1)

In [33]:
# удаление дубликатов
initial_rows = len(filtered_data)
filtered_data = filtered_data.drop_duplicates()
removed_rows = initial_rows - len(filtered_data)
print(f"Количество удаленных дубликатов: {removed_rows}")

Количество удаленных дубликатов: 8


In [34]:
columns_to_fill_zero = ['education_form', 'education_status']
filtered_data.loc[:, columns_to_fill_zero] = filtered_data[columns_to_fill_zero].fillna("Unknown")

In [35]:
filtered_data = filtered_data.astype(str)
def clean_text(text):
    # if not isinstance(text, (str, bytes)):
    #     return str(text)

    text = re.sub(r'http\S+|www\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[\n*#%\r!?)()".,-:=+]', '', text)
    text = ''.join(char for char in text if unicodedata.category(char) != 'So')
    return text

text_columns = filtered_data.select_dtypes(include='object').columns

filtered_data[text_columns] = filtered_data[text_columns].applymap(clean_text)

filtered_data = filtered_data.applymap(clean_text)

In [37]:
columns_to_replace = ['about', 'activities', 'books', 'games', "interests"]
filtered_data[columns_to_replace] = filtered_data[columns_to_replace].applymap(
    lambda x: 'нет' if (isinstance(x, str) and len(x) < 3) else x
)
filtered_data[columns_to_replace] = filtered_data[columns_to_replace].applymap(lambda x: x.lower() if isinstance(x, str) else x)
values_to_replace = [0, '', ' ', "0", "-", "nan"]
filtered_data[columns_to_replace] = filtered_data[columns_to_replace].replace(values_to_replace, "нет")

In [51]:
filtered_data.to_csv("filtered_data.csv", index=False)

In [50]:
filtered_data

,about,activities,books,games,interests,education_form,education_status,university_name,faculty_name
9,мама люба,несу людям счастье и калории,акунин бунин достоевский,догонялки,шоппинг даник кулинария,Очное отделение,Студентка бакалавр,СПбГУ,Международных отношений
43,нет,ja spb завсектором экономического моделирован...,ммм,freebsd linux apache mysql php perl html ssi w...,web design web usability seo optimization inst...,Очное отделение,Выпускник бакалавр,НИУ ВШЭ СанктПетербург,Экономики
52,хотите в болото спросите меня как,аспирант лаборантисследователь,кизи весь,nfs дальнобойщики,у много всякого вернее все подряд но в особенн...,Очное отделение,Выпускник магистр,СПбПУ Петра Великого Политех,Институт физики нанотехнологий и телекоммуникаций
70,спасибо тем кто меня любит вы делаете меня лу...,sales deadline организация промомероприятий,гришковец в остальном только периодика и проф...,deadline куб рубика кубик рубик кубик рубика p...,bmw honda management reading sleeping,Очное отделение,Студент специалист,НИУ ВШЭ СанктПетербург,Менеджмента
87,целеустремленный человек меняю мир к лучшему ч...,исследователь разработчик и популяризатор техн...,мастер и маргарита,шахматы мэком dead line,образование саморазвитие спорт достижения наук...,Очное отделение,Выпускник бакалавр,РГПУ им А И Герцена,Факультет математики
...,...,...,...,...,...,...,...,...,...
4193113,нет,нет,атлант расправил плечи финансист алхимик чайка...,монополия,самообразование книги инвестиции как стать сча...,Заочное отделение,Выпускник бакалавр,МФПУ «Синергия» бывш МФПА ММИЭИФП,Факультет психологии
4198479,я хорошего мнения,культурная,нет,нет,разносторнние,Очное отделение,Unknown,СПб НИПНИ им Бехтерева,Unknown
4202832,не бей своих чтобы боялись чужие лучше бей себ...,прыгаю и трясу конечностями типо лиза превокур...,лошадиная энциклопедия,durka simulator,я комар ты фумигатор,Unknown,Unknown,АлтГТУ им Ползунова,Unknown
4207485,чтобы сделать заказ нужно оставить свой комме...,хотите покупать стильные вещи дешевле чем в ма...,нет,нет,заказать понравившуюся вещь очень просто для э...,Очное отделение,Выпускница специалист,ЮУрГАУ бывш ЧИМЭСХ ЧГАА ЧГАУ,Факультет технического сервиса в агропромышлен...


In [40]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33759 entries, 9 to 4208359
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   about             33759 non-null  object
 1   activities        33759 non-null  object
 2   books             33759 non-null  object
 3   games             33759 non-null  object
 4   interests         33759 non-null  object
 5   education_form    33759 non-null  object
 6   education_status  33759 non-null  object
 7   university_name   33759 non-null  object
 8   faculty_name      33759 non-null  object
 9   city_title        33759 non-null  object
dtypes: object(10)
memory usage: 2.8+ MB


In [41]:
#filtered_data.reset_index(drop=True, inplace=True)

## Классификация текста (пробно)

In [42]:
from transformers import pipeline

In [43]:
zero_df = filtered_data.copy().head(10)

In [44]:
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [46]:

# books_category = [
#     "Программирование",
#     'Художественная литература',
#     "Научная литература",
#     "Фентези",
#     "Научная Фантастика",
#     "Психология",
#     "Спорт",
#     "Разное"
# ]


# for index, row in zero_df.iterrows():
#     sequence_to_classify = row["books"]


#     print(f"sequence_to_classify: {sequence_to_classify}")


#     output = classifier(sequence_to_classify, books_category)#, multi_label=True)


#     predicted_categories = [label for label, score in zip(output["labels"], output["scores"]) if score > 0.5]


#     if len(predicted_categories) > 0:
#         zero_df.at[index, "books"] = predicted_categories[0]
#     else:
#         zero_df.at[index, "books"] = None

# zero_df

In [47]:
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

In [48]:
# books_category = [
#     "Программирование",
#     'Художественная литература',
#     "Научная литература",
#     "Фентези",
#     "Научная Фантастика",
#     "Психология",
#     "Спорт",
#     "Разное"
# ]


# for index, row in zero_df.iterrows():
#     sequence_to_classify = row["books"]


#     output = classifier(sequence_to_classify, books_category, multi_label=True)


#     predicted_categories = [label for label, score in zip(output["labels"], output["scores"]) if score > 0.5]


#     if len(predicted_categories) > 0:
#         zero_df.at[index, "books"] = predicted_categories[0]
#     else:
#         zero_df.at[index, "books"] = "Разное"

# zero_df